In [ ]:
import pandas as pd
import os
from pandas import DataFrame
from pandas import concat
from sklearn import preprocessing
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import SimpleRNN, LSTM
import matplotlib.pyplot as plt

def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('scale%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('scale%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('scale%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

dataset = pd.read_csv('C:/Users/user/datafiles/input_data/data0102-1.csv', header=0, index_col=0)
values = dataset.values
Features = values
minmax_scale = preprocessing.MinMaxScaler(feature_range=(-1,1))
scaleFeature = minmax_scale.fit_transform(Features)
reframed = series_to_supervised(scaleFeature, 24, 1)
reframed.drop(reframed.columns[[6,13,20,27,34,41,48,55,62,69,76,83,90,97,104,111,118,125,132,139,146,153,160,167]], axis=1, inplace=True)

msk = numpy.random.rand(len(reframed)) < 0.8
train_df = reframed[msk]
test_df = reframed[~msk]

train_X, train_y = train_df.values[:, 0:-1], train_df.values[:, -1]
test_X, test_y = test_df.values[:, 0:-1], test_df.values[:, -1]

train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1],train_X.shape[2]), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=1))
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

train_history = model.fit(train_X , train_y, validation_data=(test_X, test_y), batch_size=25,
                          shuffle=True,epochs=10, verbose=2)

scores = model.evaluate(test_X, test_y)
scores[1]

show_train_history(train_history, 'acc', 'val_acc')
show_train_history(train_history, 'loss', 'val_loss')